# Rllib
the purpose of this jupyter notebook is to have preliminary understanding of how Rllib is structured and working
it is based on [their site](https://ray.readthedocs.io/en/latest/rllib.html)

![image of rllib stack](rllib-stack.svg)

After installing ray\[rllib\] we can run it in either of 2 ways:

In [ ]:
# through shell command line
! rllib train --run=PPO --env=CartPole-v0  # -v [-vv] for verbose,
                                         # --eager [--trace] for eager execution,
                                         # --torch to use PyTorch

In [1]:
# with python API (using tune)
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
tune.run(PPOTrainer, config={"env": "CartPole-v0"})  # "log_level": "INFO" for verbose,
                                                     # "eager": True for eager execution,
                                                     # "torch": True for PyTorch

2020-03-08 21:59:50,614	WARNING services.py:586 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-03-08 21:59:50,616	INFO resource_spec.py:212 -- Starting Ray with 13.62 GiB memory available for workers and up to 6.82 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-08 21:59:50,958	INFO services.py:1078 -- View the Ray dashboard at localhost:8265


Trial name,status,loc
PPO_CartPole-v0_00000,RUNNING,


2020-03-08 21:59:52,369	WARNING worker.py:1058 -- The dashboard on node guy-970 failed with the following error:
Traceback (most recent call last):
  File "/home/guy/anaconda3/envs/rlib20/lib/python3.7/site-packages/ray/dashboard/dashboard.py", line 918, in <module>
    redis_password=args.redis_password,
  File "/home/guy/anaconda3/envs/rlib20/lib/python3.7/site-packages/ray/dashboard/dashboard.py", line 134, in __init__
    self.setup_routes()
  File "/home/guy/anaconda3/envs/rlib20/lib/python3.7/site-packages/ray/dashboard/dashboard.py", line 333, in setup_routes
    build_dir)
FileNotFoundError: [Errno 2] Dashboard build directory not found. If installing from source, please follow the additional steps required to build the dashboard (cd python/ray/dashboard/client && npm ci && npm run build): '/home/guy/anaconda3/envs/rlib20/lib/python3.7/site-packages/ray/dashboard/client/build'



(pid=12071) 2020-03-08 21:59:53,991	INFO trainer.py:423 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
(pid=12071) 2020-03-08 21:59:53,992	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=12071) 2020-03-08 21:59:57,200	WARNING util.py:37 -- Install gputil for GPU system monitoring.
Result for PPO_CartPole-v0_00000:
  custom_metrics: {}
  date: 2020-03-08_22-00-03
  done: false
  episode_len_mean: 21.884615384615383
  episode_reward_max: 56.0
  episode_reward_mean: 21.884615384615383
  episode_reward_min: 9.0
  episodes_this_iter: 182
  episodes_total: 182
  experiment_id: 3ed2ab9e0aea41c1ad8d62cf7e422c0b
  experiment_tag: '0'
  hostname: guy-970
  info:
    grad_time_ms: 2956.736
    learner:
      default_policy:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 4.999999873689376e-05
        entropy: 0.6622737050056458
        entropy_coeff: 0.0
      

Trial name,status,loc,reward,total time (s),ts,iter
PPO_CartPole-v0_00000,RUNNING,192.168.1.93:12071,21.8846,5.96005,4000,1


...

Result for PPO_CartPole-v0_00000:
  custom_metrics: {}
  date: 2020-03-08_22-01-59
  done: false
  episode_len_mean: 199.16
  episode_reward_max: 200.0
  episode_reward_mean: 199.16
  episode_reward_min: 185.0
  episodes_this_iter: 20
  episodes_total: 805
  experiment_id: 3ed2ab9e0aea41c1ad8d62cf7e422c0b
  experiment_tag: '0'
  hostname: guy-970
  info:
    grad_time_ms: 2677.681
    learner:
      default_policy:
        cur_kl_coeff: 0.0023437500931322575
        cur_lr: 4.999999873689376e-05
        entropy: 0.46547481417655945
        entropy_coeff: 0.0
        kl: 0.005870374385267496
        policy_loss: -0.000582060485612601
        total_loss: 138.92674255371094
        vf_explained_var: 0.7581650614738464
        vf_loss: 138.9273223876953
    load_time_ms: 1.551
    num_steps_sampled: 108000
    num_steps_trained: 107136
    sample_time_ms: 1752.359
    update_time_ms: 4.795
  iterations_since_restore: 27
  node_ip: 192.168.1.93
  num_healthy_workers: 2
  off_policy_estimato

Trial name,status,loc,reward,total time (s),ts,iter
PPO_CartPole-v0_00000,RUNNING,192.168.1.93:12071,199.16,121.466,108000,27


Result for PPO_CartPole-v0_00000:
  custom_metrics: {}
  date: 2020-03-08_22-02-08
  done: false
  episode_len_mean: 198.4
  episode_reward_max: 200.0
  episode_reward_mean: 198.4
  episode_reward_min: 176.0
  episodes_this_iter: 20
  episodes_total: 846
  experiment_id: 3ed2ab9e0aea41c1ad8d62cf7e422c0b
  experiment_tag: '0'
  hostname: guy-970
  info:
    grad_time_ms: 2676.752
    learner:
      default_policy:
        cur_kl_coeff: 0.0023437500931322575
        cur_lr: 4.999999873689376e-05
        entropy: 0.46278899908065796
        entropy_coeff: 0.0
        kl: 0.004252588376402855
        policy_loss: -0.00031634251354262233
        total_loss: 197.55064392089844
        vf_explained_var: 0.6675523519515991
        vf_loss: 197.5509490966797
    load_time_ms: 1.536
    num_steps_sampled: 116000
    num_steps_trained: 115072
    sample_time_ms: 1747.086
    update_time_ms: 4.786
  iterations_since_restore: 29
  node_ip: 192.168.1.93
  num_healthy_workers: 2
  off_policy_estimato

Trial name,status,loc,reward,total time (s),ts,iter
PPO_CartPole-v0_00000,RUNNING,192.168.1.93:12071,198.4,130.288,116000,29


KeyboardInterrupt: 

## Key concepts in Rllib
There are 3 key concepts: Policies, Samples and Trainers

### Policies
[Policies](https://ray.readthedocs.io/en/latest/rllib-concepts.html#policies) are Python classes that define how an agent acts in an environment. [Rollout workers](https://ray.readthedocs.io/en/latest/rllib-concepts.html#policy-evaluation) query the policy to determine agent actions. In a gym environment, there is a single agent and policy. In [vector envs](https://ray.readthedocs.io/en/latest/rllib-env.html#vectorized), policy inference is for multiple agents at once, and in [multi-agent](https://ray.readthedocs.io/en/latest/rllib-env.html#multi-agent-and-hierarchical), there may be multiple policies, each controlling one or more agents.  
Policies can be implemented using any framework. However, for TensorFlow and PyTorch, RLlib has [build_tf_policy](https://ray.readthedocs.io/en/latest/rllib-concepts.html#building-policies-in-tensorflow) and [build_torch_policy](https://ray.readthedocs.io/en/latest/rllib-concepts.html#building-policies-in-pytorch) helper functions that let you define a trainable policy with a functional-style API, for example:

In [2]:
def policy_gradient_loss(policy, model, dist_class, train_batch):
    logits, _ = model.from_batch(train_batch)
    action_dist = dist_class(logits, model)
    return -tf.reduce_mean(
        action_dist.logp(train_batch["actions"]) * train_batch["rewards"])

# <class 'ray.rllib.policy.tf_policy_template.MyTFPolicy'>
MyTFPolicy = build_tf_policy(
    name="MyTFPolicy",
    loss_fn=policy_gradient_loss)

NameError: name 'build_tf_policy' is not defined

### Sample Batches
Whether running in a single process or [large cluster](https://ray.readthedocs.io/en/latest/rllib-training.html#specifying-resources), all data interchange in RLlib is in the form of [sample batches](https://github.com/ray-project/ray/blob/master/rllib/policy/sample_batch.py). Sample batches encode one or more fragments of a trajectory. Typically, RLlib collects batches of size `sample_batch_size` from rollout workers, and concatenates one or more of these batches into a batch of size `train_batch_size` that is the input to SGD.  
A typical sample batch looks something like the following when summarized. Since all values are kept in arrays, this allows for efficient encoding and transmission across the network:

```
{ 'action_logp': np.ndarray((200,), dtype=float32, min=-0.701, max=-0.685, mean=-0.694),
  'actions': np.ndarray((200,), dtype=int64, min=0.0, max=1.0, mean=0.495),
  'dones': np.ndarray((200,), dtype=bool, min=0.0, max=1.0, mean=0.055),
  'infos': np.ndarray((200,), dtype=object, head={}),
  'new_obs': np.ndarray((200, 4), dtype=float32, min=-2.46, max=2.259, mean=0.018),
  'obs': np.ndarray((200, 4), dtype=float32, min=-2.46, max=2.259, mean=0.016),
  'rewards': np.ndarray((200,), dtype=float32, min=1.0, max=1.0, mean=1.0),
  't': np.ndarray((200,), dtype=int64, min=0.0, max=34.0, mean=9.14)}```

### Training
Policies each define a `learn_on_batch()` method that improves the policy given a sample batch of input. For TF and Torch policies, this is implemented using a loss function that takes as input sample batch tensors and outputs a scalar loss. Here are a few example loss functions:  
- Simple [policy gradient loss](https://github.com/ray-project/ray/blob/master/rllib/agents/pg/pg_tf_policy.py)
- Simple [Q-function loss](https://github.com/ray-project/ray/blob/a1d2e1762325cd34e14dc411666d63bb15d6eaf0/rllib/agents/dqn/simple_q_policy.py#L136)
- Importance-weighted [APPO surrogate loss](https://github.com/ray-project/ray/blob/master/rllib/agents/ppo/appo_policy.py)  

RLlib [Trainer classes](https://ray.readthedocs.io/en/latest/rllib-concepts.html#trainers) coordinate the distributed workflow of running rollouts and optimizing policies. They do this by leveraging [policy optimizers](https://ray.readthedocs.io/en/latest/rllib-concepts.html#policy-optimization) that implement the desired computation pattern. The following figure shows synchronous sampling, the simplest of [these patterns](https://ray.readthedocs.io/en/latest/rllib-algorithms.html):  

![a2c-arch](a2c-arch.svg)  


RLlib uses [Ray actors](https://ray.readthedocs.io/en/latest/actors.html) to scale training from a single core to many thousands of cores in a cluster. You can [configure the parallelism](https://ray.readthedocs.io/en/latest/rllib-training.html#specifying-resources) used for training by changing the `num_workers` parameter.

### Customization
RLlib provides ways to customize almost all aspects of training, including the [environment](https://ray.readthedocs.io/en/latest/rllib-env.html#configuring-environments), [neural network model](https://ray.readthedocs.io/en/latest/rllib-models.html#tensorflow-models), [action distribution](https://ray.readthedocs.io/en/latest/rllib-models.html#custom-action-distributions), and [policy definitions](https://ray.readthedocs.io/en/latest/rllib-concepts.html#policies):
![rllib_components](rllib-components.svg)

# Conecpts and Custom Algorithms
The following is selected items taken from the [documentation of rllib](https://ray.readthedocs.io/en/latest/rllib-toc.html)

## [Policies](https://ray.readthedocs.io/en/latest/rllib-concepts.html#policies)
This section describes the internal concepts used to implement algorithms in RLlib. You might find this **useful if modifying or adding new algorithms to RLlib.**  
Policy classes encapsulate the core numerical components of RL algorithms. typically includes:
 - Policy model that determines actions to take
 - A trajectory postprocessor for experiences
 - loss function to improve the policy given postprocessed experiences  

for simple example, see the PG [policy definition](https://github.com/ray-project/ray/blob/master/rllib/agents/pg/pg_tf_policy.py)  

Most of the interaction with deep learning is isolated to the [Policy interface](https://github.com/ray-project/ray/blob/master/rllib/policy/policy.py) allowing RLlib to support multiple frameworks. there are [Tensorflow](https://ray.readthedocs.io/en/latest/rllib-concepts.html#building-policies-in-tensorflow) and [PyTorch](https://ray.readthedocs.io/en/latest/rllib-concepts.html#building-policies-in-pytorch) specific templates.  
You can write your own from scratch as follows:

In [ ]:
class CustomPolicy(Policy):
    """Example of a custom policy written from scratch.

    You might find it more convenient to use the `build_tf_policy` and
    `build_torch_policy` helpers instead for a real policy, which are
    described in the next sections.
    """

    def __init__(self, observation_space, action_space, config):
        Policy.__init__(self, observation_space, action_space, config)
        # example parameter
        self.w = 1.0

    def compute_actions(self,
                        obs_batch,
                        state_batches,
                        prev_action_batch=None,
                        prev_reward_batch=None,
                        info_batch=None,
                        episodes=None,
                        **kwargs):
        # return action batch, RNN states, extra values to include in batch
        return [self.action_space.sample() for _ in obs_batch], [], {}

    def learn_on_batch(self, samples):
        # implement your learning code here
        return {}  # return stats

    def get_weights(self):
        return {"w": self.w}

    def set_weights(self, weights):
        self.w = weights["w"]

For using policy abstraction in multi agent, see the [rock-paper-scisors example](https://ray.readthedocs.io/en/latest/rllib-env.html#rock-paper-scissors-example)

### Building policies in Tensorflow
describes how to build a tensorflow RLlib policy using `tf_policy_template.build_tf_policy()`

#### Define the loss function
In RLlib, loss functions are defined over batches of trajectory data produced by policy evaluation. A basic policy gradient loss that only tries to maximize the 1-step reward can be defined as follows:

In [ ]:
import tensorflow as tf
from ray.rllib.policy.sample_batch import SampleBatch

def policy_gradient_loss(policy, model, dist_class, train_batch):
    actions = train_batch[SampleBatch.ACTIONS]
    rewards = train_batch[SampleBatch.REWARDS]
    logits, _ = model.from_batch(train_batch)
    action_dist = dist_class(logits, model)
    return -tf.reduce_mean(action_dist.logp(actions) * rewards)

where `actions` is a Tensor placeholder of shape \[batch_size, action_dim...\], and `rewards` is a placeholder of shape \[batch_size\].  
The `action_dist` object is an [ActionDistribution](https://ray.readthedocs.io/en/latest/rllib-package-ref.html#ray.rllib.models.ActionDistribution) that is parameterized by the output of the neural network policy model. Passing this loss function to `build_tf_policy` is enough to produce a very basic TF policy:

In [ ]:
from ray.rllib.policy.tf_policy_template import build_tf_policy

# <class 'ray.rllib.policy.tf_policy_template.MyTFPolicy'>
MyTFPolicy = build_tf_policy(
    name="MyTFPolicy",
    loss_fn=policy_gradient_loss)

as an exercise (runnable file [here](https://github.com/ray-project/ray/blob/master/rllib/examples/custom_tf_policy.py)) we can create a [Trainer](https://ray.readthedocs.io/en/latest/rllib-concepts.html#trainers)  and try running this policy on a toy env with two parallel rollout workers:

In [ ]:
import ray
from ray import tune
from ray.rllib.agents.trainer_template import build_trainer

# <class 'ray.rllib.agents.trainer_template.MyCustomTrainer'>
MyTrainer = build_trainer(
    name="MyCustomTrainer",
    default_policy=MyTFPolicy)

ray.init()
tune.run(MyTrainer, config={"env": "CartPole-v0", "num_workers": 2})